# Microbatching

## What's the point of microbathing?

As we know, bigger batch size allows for better gradients estimates, thus helping training models. Unfortunately, there is a hard constraint on device memory: even the modern accelerators can't fit more that hundreds of gigabytes, which is sometimes just not enough. This is where `microbatch` comes to the rescue: it allows to evenly split batch data into multiple pieces (called microbatches), evaluate gradients on each of them separately, and the apply average value of gradient directly to the weights of the model.

Parameter `microbatch` allows us to leverage simple trade-off between bigger batch size (thus model performance) and model training time.

In [1]:
import os
import sys
import warnings

import tensorflow as tf

sys.path.append('../../..')
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import Imagenette320
from batchflow.models.tf import ResNet18

Specify which GPU(s) to be used. More about it in [CUDA documentation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars).

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


## Create a dataset, define a default model config

In [ ]:
dataset = Imagenette320(bar=True)

model_config = {'inputs/images/shape': B.image_shape,
                'inputs/labels/classes': D.num_classes,
                'initial_block/inputs': 'images'}

BATCH_SIZE = 4160

  0%|          | 0/2 [00:00<?, ?it/s]

# Train model without microbatch

In [ ]:
train_template = (Pipeline()
                  .init_variable('loss_history', [])
                  .init_model('dynamic', ResNet18, 'conv_nn', config=model_config)
                  .resize((320, 320))
                  .to_array()
                  .train_model('conv_nn', fetches='loss',
                               images=B.images, labels=B.labels,
                               save_to=V('loss_history', mode='a')))

In [ ]:
train_pipeline = train_template << dataset.train
try:
    train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True, drop_last=True)
except tf.errors.ResourceExhaustedError:
    print('ResourceExhaustedError')

We get **ResourceExhaustedError** because batch didn't fit into GPU.

# Add microbatch

We can add `microbatch` to model configuration:

In [4]:
model_config.update({'microbatch': 64})

Now, batches will be split into microbatches with size 64 inside the [TFModel.train](https://analysiscenter.github.io/batchflow/api/batchflow.models.tf.base.html#batchflow.models.tf.base.TFModel.train) method.

> **MICROBATH SIZE MUST BE A DIVISOR OF THE BATCH SIZE!**

# Train model with microbatch

In [5]:
template_microbatch = (Pipeline()
                       .init_variable('loss_history', [])
                       .init_model('dynamic', ResNet18, 'conv_nn', config=model_config)
                       .resize((320, 320))
                       .to_array()
                       .train_model('conv_nn', fetches='loss',
                                    images=B.images, labels=B.labels,
                                    save_to=V('loss_history', mode='a')))

In [6]:
pipeline_microbatch = template_microbatch << dataset.train
pipeline_microbatch.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True, drop_last=True)

100%|██████████| 3/3 [02:06<00:00, 45.82s/it]


If you didn’t have `microbatch` in the model configuration and can also pass `microbatch` parameter to the `train_model` action of pipeline to dynamically change splitting strategy. You could run pipeline with parameter `train_model` parameter `microbatch=microbatch_size`:
```python
train_model('conv_nn', fetches='loss',
            microbatch=64,
            images=B.images, labels=B.labels,
            save_to=V('loss_history', mode='a'))
```
If you defined `microbatch`  in both places (model configuration and train model action), then value in `train_model` will be used.

 **If the data in batch can fit memory constraints, there is no reason to use `microbatch` due to inherently slower processing of batches.**

Now we can train models using `microbatch` and you might want to see next tutorial about [multiple devices](./02_device.ipynb).